# Electrometer Data Viewer

In [ ]:
# User input 
dayobs = 20250910
groupid = 'BT529_O_20250910_000001_106858_001#1'

In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
import datetime as dt

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from astropy.io import fits
from astropy.table import Table
from astropy.time import Time, TimeDelta

# LSST-specific packages
from lsst.resources import ResourcePath
from lsst_efd_client import EfdClient

In [ ]:
dayobs_str = str(dayobs)
iso_str = f"{dayobs_str[:4]}-{dayobs_str[4:6]}-{dayobs_str[6:]}"

start_time = Time(iso_str, format="iso")
end_time = start_time + TimeDelta(60*60*24,format='sec')

client = EfdClient('usdf_efd')
selections = ["url","id"]
msg_log_topic = "lsst.sal.Electrometer.logevent_largeFileObjectAvailable"

elec_df = await client.select_time_series(msg_log_topic,selections, start=start_time, end=end_time)

In [ ]:
hdr = None
try:
    this_obs = elec_df[elec_df.id == groupid]
    url = this_obs.url.values[0]
    path = ResourcePath('s3://lfa@' + url.split('.org/')[1])
    with path.open("rb") as f:
        hdu = fits.open(f)
        hdr = hdu[0].header
        fig,ax = plt.subplots()
        ax.plot(hdu[1].data['Elapsed Time'], hdu[1].data['Signal'], 'x-', label=f"Mean (std) {np.mean(hdu[1].data['Signal']):.2e} ({np.std(hdu[1].data['Signal']):.2e})")
        ax.set_title(f"Electrometer Data: {groupid}")
        ax.set_xlabel("Exposure Time (s)")
        ax.set_ylabel("Photodiode Current")
        ax.legend()
except Exception as e:
    print(f"The groupid you input doesn't seem to be available: {e}")

In [ ]:
hdr